In [21]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")
# Specify the path to your existing CSV file
csv_file_path = 'amazon_validation9.csv'

# Read ASINs from 'ASINS.csv'
asins_file_path = 'ASINs_validation-CA.csv'
with open(asins_file_path, 'r') as asins_file:
    # Assuming ASINs are in the first column of the CSV file
    #asins_list = [row[0] for row in csv.reader(asins_file)]
    # Assuming ASINs are in the first column of the CSV file
    asins_list = [row[0] for row in csv.reader(asins_file) if row]

# Create a list to store failed ASINs
failed_asins_list = []

# Open the CSV file in append mode
with open(csv_file_path, 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)

    # Check if the file is empty and write headers if needed
    if f.tell() == 0:
        # writer.writerow(['ASIN', 'Title','products_overview','bullet_points','product_information','product_description'])
        writer.writerow(['ASIN', 'Title','buybox_owner','Price','products_overview','bullet_point','product_information','product_description','Review_check','image_block','Bread_crumb'])

    # Iterate through each ASIN and fetch data simultaneously
    for asin in asins_list:
        retry_count = 2  # Number of retries
        while retry_count > 0:
            # Create the URL for the current ASIN
            base_url = "https://www.amazon.ca/dp/"
            url = base_url + asin

            # Make a request to the Amazon product page
            headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Referrer Policy": "strict-origin-when-cross-origin",
    "Accept-Ch": "ect,rtt,downlink,device-memory,sec-ch-device-memory,viewport-width,sec-ch-viewport-width,dpr,sec-ch-dpr,sec-ch-ua-platform,sec-ch-ua-platform-version",
    "Accept-Ch-Lifetime": "86400",
    "content-type": "text/html;charset=UTF-8",
    "Alt-Svc": "h3=\":443\"; ma=86400",
    "Accept-Encoding": "gzip, deflate,br",
    "x-amz-rid": "HQJ3W1FBJAGR7RX433F8",
    "authority": "www.amazon.ca",
    "Request Method": "GET",
    "path": "/vap/ew/componentbuilder",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1",
    "Cookie": """session-id=135-2523829-3628852; ubid-acbca=133-8901075-5009164; lc-acbca=en_CA; i18n-prefs=CAD; at-acbca=Atza|IwEBIN96ZVnwpB3wtunoui1XN6sZjOnzwM4eHpW2HTe95_sPToQkSj8f2z9s0D3eR9b6yxZG8_vrE9SOO_Xx4cr2P1VCp44JnORPPHtEdNRmOlh4Oz9eioNfyxyCN2mbZjvOmMjgh4VUs20jbeODDYmDi6GCWEViYEUCbP7OtYLcRRRhxvX_e4ElutOj5F4ZVE7tYvgsQVuCaRuRWpBmYym2eGGj4EM6J0T01oEH0cBNJM_zOSeK8FiT1w08E_W_VT1w6b1u1bYpJh4rsNj3mJP07AltGX6117m2RKNYIxlo3AWfZg; sess-at-acbca="k+omQrmizMOAQCUSS6STg4hcQo9zDW/sm7ny773cKVc="; session-id-time=2082787201l; session-token=SpYzn7og6U72Ztc4PcFW8ld9Tr4P9YGkNcfzKf6kWNaJYn2dl1Dvv9r92X6p6fvC7PEbwz3X7bVkq6gqMzdQKZOcpWhfbnqU8YZKErw93D6z9db4KMMSkRy1kMt3Vu87L382PWWyTPg2YwFe7zy91rcC6whMPSgUl82mxn/Yd87DCmjB+HZ16i1OWbH5iOr0vuuaymJ7Dqv9uzNIXZe81rlrGsapdseIN5dHyZVILGslxtFUNytswXOKHS7A5Y2706nJ/KHuO4uUHwp4jS+JnzepIbsyHY1fGKdUBlhxJmnDiOGk7qkNyCo9/nvz3Ro5S2gfu943FcO0Lyu+krFoRRkql9ZccV5/2CtQzjzfcfm/5PNSdvXtncR4K9KEquqN; x-acbca="s16dBUWSDVD7YMIITg0E0ZytUcBlV625NdSUk3KWYykA?wl?Md0tccDqzJlHgiPH"; csm-hit=tb:PE8ABJ44J1B7AH0BYVG6+s-1YK9Y940XDMRHXQQ61B0|1738144518144&t:1738144518144&adb:adblk_no"""
}
            try:
                page = requests.get(url, headers=headers)
                time.sleep(0.8)  # Introduce a delay between requests
            except Exception as e:
                print(f"Error fetching data for ASIN {asin}: {e}")
                # Append the ASIN to the failed_asins_list
                failed_asins_list.append({'ASIN': asin, 'Reason': str(e)})
                continue  # Retry if there is an error

            # Parse the HTML content
            soup1 = BeautifulSoup(page.content, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            # Extract data from the page
            title = soup2.find(id='productTitle').get_text().strip() if soup2.find(id='productTitle') else 'N/A'
            Bread_crumb = soup2.find(id='wayfinding-breadcrumbs_feature_div').get_text().strip() if soup2.find(
                id='wayfinding-breadcrumbs_feature_div') else 'N/A'
            buybox_owner = soup2.find(id= "offer-display-features") if soup2.find(id= "offer-display-features") else 'N/A'
            price = 'N/A'
            unavailable_count = soup2.get_text().count("Currently unavailable")            

            # Check if the item is unavailable
            if unavailable_count >= 2:
                price = 'N/A'
                print(f"ASIN {asin} is unavailable. Setting price to 'N/A'.")
            else:
                # Attempt to find price in various sections
                if price == 'N/A':
                    print("Attempting to find price in multiple sections.")
                    
                    # List of potential sections with price spans
                    potential_ids = [
                        'corePriceDisplay_desktop_feature_div',  # Main target
                        'priceblock_ourprice',                  # Fallback for list price
                        'priceblock_dealprice',                 # For deals
                        'corePrice_feature_div',
                        'aok-offscreen'
                                                    # Another possible section
                    ]
                    
                    price_section = None
                    for section_id in potential_ids:
                        price_section = soup2.find('div', id=section_id)
                        if price_section:
                            break
                    
                    if not price_section:
                        # Check for price in table structure with 'Bundle Price' and 'Bundle List Price'
                        table_section = soup2.find('table', class_='a-lineitem a-align-top')
                        if table_section:
                            # Extract 'Bundle List Price'
                            bundle_list_price_span = table_section.find('span', class_='a-price a-text-price a-size-base')
                            if bundle_list_price_span:
                                bundle_list_price = bundle_list_price_span.find('span', class_='a-offscreen')
                                if bundle_list_price:
                                    price = bundle_list_price.get_text().strip()
                                    print(f"Bundle List Price found: {price}")
                            
                            # Extract 'Bundle Price'
                            if price == 'N/A':  # Only search if Bundle List Price is not found
                                bundle_price_span = table_section.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay')
                                if bundle_price_span:
                                    bundle_price = bundle_price_span.find('span', class_='a-offscreen')
                                    if bundle_price:
                                        price = bundle_price.get_text().strip()
                                        print(f"Bundle Price found: {price}")
                    
                    # Broader search for any price element (fallback)
                    if price == 'N/A':
                        print("Attempting to find price in any span with 'a-price' class.")
                        price_spans = soup2.find_all('span', class_='a-price')
                        for price_span in price_spans:
                            offscreen_price = price_span.find('span', class_='a-offscreen')
                            if offscreen_price and offscreen_price.get_text().strip():
                                price = offscreen_price.get_text().strip()
                                print(f"Fallback price found: {price}")
                                break  # Exit loop once a valid price is found            

                    if price_section:
                        # Search for the price in the section
                        price_block = price_section.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
                        if price_block:
                            price_symbol = price_block.find('span', class_='a-price-symbol')
                            price_whole = price_block.find('span', class_='a-price-whole')
                            price_fraction = price_block.find('span', class_='a-price-fraction')            

                            if price_symbol and price_whole and price_fraction:
                                price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
                                print(f"Price found: {price}")
                            else:
                                print("Price components not found in the specified structure.")
                        else:
                            print("Price block not found in the identified section.")
                    else:
                        print("No relevant price section found.")
            products_overview = soup2.find('div', {'data-csa-c-content-id':'productOverview'}) 
            #product_overview = soup2.find(id='productOverview_feature_div').get_text().strip() if soup2.find(id='productOverview_feature_div') else 'N/A'
            bullet_point_element = soup2.find(id='productFactsDesktopExpander') or soup2.find(id='feature-bullets')
            bullet_point = bullet_point_element.get_text().strip() if bullet_point_element else 'N/A'
            product_information = soup2.find(id='prodDetails').get_text().strip() if soup2.find(id='prodDetails') else 'N/A'
            product_description = soup2.find(id='productDescription').get_text().strip() if soup2.find(id='productDescription') else 'N/A'
            Review_check = soup2.find(id='reviewsMedley').get_text().strip() if soup2.find(
                id='reviewsMedley') else 'N/A'
                        # Locate the #imageBlock_feature_div section
            image_block_feature_div = soup2.find("div", id="imageBlock_feature_div")            

             # Locate the desired script within the #imageBlock_feature_div
            image_block_script = (
                    image_block_feature_div.find("script", text=lambda t: t and "P.when('A').register(\"ImageBlockATF\"" in t)
                    if image_block_feature_div
                    else None
            )
            image_block = image_block_script.string.strip() if image_block_script else 'N/A'
            # image_block = image_block_script.text.strip() if image_block_script else 'N/A'
            # imgs_str = img_div.img.get('data-a-dynamic-image')  # a string in Json format
            # Reviews_overall = soup2.find(id='customerReviews').get_text().strip() if soup2.find(
            #    id='customerReviews') else 'N/A'
            # Insights_reviews = soup2.find(id='cr-product-insights-cards').get_text().strip() if soup2.find(
            #    id='cr-product-insights-cards') else 'N/A'
            # Twister_data = soup2.find(id='twister').get_text().strip() if soup2.find(
            #    id='twister') else 'N/A'
                

            # Check if title or ratings is 'N/A' and retry if needed
            if title == 'N/A':
                retry_count -= 1
                print(f"Retrying for ASIN {asin}. Retries left: {retry_count}")
            else:
                break  # Break out of the retry loop if successful

        # If all retries failed, print an error message
        if retry_count == 0:
            print(f"Failed to fetch data for ASIN {asin} after multiple retries.")
            # Append the ASIN to the failed_asins_list
            failed_asins_list.append({'ASIN': asin, 'Reason': 'Max retries reached'})
            continue

        # Write the data to the CSV file
        writer.writerow([asin, title,buybox_owner,price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb])
        # writer.writerow([asin, title, price])

# Convert the failed_asins_list to a DataFrame
failed_asins_df = pd.DataFrame(failed_asins_list)

# Save the failed ASINs to a separate CSV file
failed_asins_df.to_csv('failed_asins-final.csv', index=False)

Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ﻿B08KWSYJ4F. Retries left: 1
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ﻿B08KWSYJ4F. Retries left: 0
Failed to fetch data for ASIN ﻿B08KWSYJ4F after multiple retries.


In [22]:
print(soup2.prettify())  # After parsing


In [3]:
import pandas as pd
validation_df1=pd.read_csv('amazon_validation1.csv')

pd.set_option("display.max_columns", None)
validation_df1


,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B00DOOFTAU,"Tall Quadrato Cup, Tall Dessert Cup, Tall Appe...","<div class=""a-section a-spacing-base offer-dis...",$76\n \n ..10,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Made from premium plastic This big brother of ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1,B00NJSYCMM,Restaurantware 50 Count Bamboo Leaf Plate with...,"<div class=""a-section a-spacing-base offer-dis...",$198\n \n ..12,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Bamboo Leaf Plate with Cup Holders come ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
2,B00DOOCR7I,Restaurantware Bamboo Cone Large 200 count box,"<div class=""a-section a-spacing-base offer-dis...",$181\n \n ..44,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Large Bamboo Cone are shipped in a 200 c...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B007Y1LJIU,Restaurantware Girata Dessert Dish 100 count b...,"<div class=""a-section a-spacing-base offer-dis...",$111\n \n ..47,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,"These 3 ounce, Girata Dessert Dishes come in a...",Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B00DOOD7W2,"Pinewood Appetizer Cup, Mini Wooden Cup, Pine ...","<div class=""a-section a-spacing-base offer-dis...",$163\n \n ..96,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Shaved Pinewood Cups come in a 200 count...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
...,...,...,...,...,...,...,...,...,...,...,...
1847,B0DMSKJM5D,Restaurantware-Clean Tek Professional 11.8 x 1...,"<div class=""a-section a-spacing-base offer-dis...",$32\n \n ..75,"<div class=""celwidget"" data-csa-c-asin=""B0DMSK...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
1848,B0DSWL95DD,Restaurantware-Eco Tek 6.5 x 6.5 x 2 Inch Clam...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DSWL...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\...
1849,B0DSWJPCZM,Restaurantware-Eco Tek 9.25 x 9.25 x 2 Inch Pi...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DSWJ...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\...
1850,B0DP46Z6CQ,Restaurantware-Cater Tek 8.5 Inch Large Servin...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DP46...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...


In [7]:
import pandas as pd
validation_df2=pd.read_csv('amazon_validation2.csv')

pd.set_option("display.max_columns", None)
validation_df2

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B00DOOFTAU,"Tall Quadrato Cup, Tall Dessert Cup, Tall Appe...","<div class=""a-section a-spacing-base offer-dis...",$76\n \n ..10,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Made from premium plastic This big brother of ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1,B00NJSYCMM,Restaurantware 50 Count Bamboo Leaf Plate with...,"<div class=""a-section a-spacing-base offer-dis...",$197\n \n ..11,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Bamboo Leaf Plate with Cup Holders come ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
2,B00DOOCR7I,Restaurantware Bamboo Cone Large 200 count box,"<div class=""a-section a-spacing-base offer-dis...",$175\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Large Bamboo Cone are shipped in a 200 c...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B007Y1LJIU,Restaurantware Girata Dessert Dish 100 count b...,"<div class=""a-section a-spacing-base offer-dis...",$111\n \n ..47,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,"These 3 ounce, Girata Dessert Dishes come in a...",Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B00DOOD7W2,"Pinewood Appetizer Cup, Mini Wooden Cup, Pine ...","<div class=""a-section a-spacing-base offer-dis...",$178\n \n ..20,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These Shaved Pinewood Cups come in a 200 count...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
...,...,...,...,...,...,...,...,...,...,...,...
2095,B0DNK5JLL8,"Restaurantware-Met Lux 17 Ounce Mason Jars, 2 ...",NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DNK5...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",NaN
2096,B0DP34278L,"Restaurantware-Cater Tek 10 Pound Ice Buckets,...","<div class=""a-section a-spacing-base offer-dis...",$148\n \n ..24,"<div class=""celwidget"" data-csa-c-asin=""B0DP34...",ENHANCED INSULATION: These ice buckets for par...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2097,B0DQY82HJP,Restaurantware-LID ONLY: Clean Tek Professiona...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DQY8...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\...
2098,B0DNKB9995,Restaurantware-Cater Tek 51 Ounce Serving Bowl...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DNK8...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",NaN


In [8]:
import pandas as pd

# List of CSV files to combine
files = ['amazon_validation1.csv','amazon_validation2.csv']

# Initialize an empty list to hold the DataFrames
dfs = []

# Loop over the list of files and read each one into a DataFrame
for filename in files:
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
validation_df = pd.concat(dfs, ignore_index=True)

# Write the combined DataFrame to a new CSV file
validation_df.to_csv('combined.csv', index=False)

print("CSV files combined successfully!")

CSV files combined successfully!


In [13]:
ASINs=validation_df.ASIN
ASINs.to_excel('asins_to_check.xlsx',index=False)

In [1]:
# Cleaning bullet pointsm

In [ ]:
import pandas as pd

# Define the function to process bullet points, excluding specific phrases
def extract_bullet_points(text):
    # Define the list of terms to exclude from bullet points
    exclusions = [
        'About this item', 'Nullify', '›', 'See more product details', 'Show more', 'Product details', 'Origin', 
        'Imported', 'Country of Origin', 'China', 'Fabric type', 'Non-woven', 'Care instructions', 'Hand Wash Only', 
        'Made in the USA and Imported', 'Fit type', 'Stretch', 'Closure type', 'Elastic', 'Pull-On', 'Tie Strap', 
        'Solid', 'Pattern', 'Nonwoven', 'Hybrid Plastic', 'Hand Wash', 'Polyester', 'Machine Wash', 'Snap','Eco-friendly', 'Wool',
        '100% Polyester', 'Polyeter','Paper','Kraft Paper','Kraft,Straw','Plastic','Vinyl','Spandex','Microfiber','Nitrile','Pva Microfiber','Upscale and Stylish!','nonwoven','Straw','Leather','Micro Fibre',
    ]
    
    # Process the text to extract and clean bullet points
    if pd.notna(text):
        # Split the text by newlines and strip any leading/trailing spaces
        lines = text.split('\n')
        bullet_points = [line.strip() for line in lines if line.strip()]
        
        # Exclude the first element, if necessary
        if len(bullet_points) > 1:
            bullet_points = bullet_points[1:]  # Exclude the first bullet point
            
        # Filter out any lines containing excluded phrases
        cleaned_bullet_points = [
            sentence for sentence in bullet_points 
            if sentence not in exclusions and not any(exclude in sentence for exclude in exclusions)
        ]
        
        return cleaned_bullet_points
    else:
        return []

# Apply the function to the DataFrame column
validation_df['Cleaned_Bullet_Points'] = validation_df['bullet_point'].apply(extract_bullet_points)

# Display the result
print(validation_df['Cleaned_Bullet_Points'])


In [ ]:
# Identify the length of each row in the 'Cleaned_Bullet_Points' column
validation_df['Length_of_Cleaned_Bullet_Points'] = validation_df['Cleaned_Bullet_Points'].apply(len)
validation_df

In [ ]:
validation_df.Cleaned_Bullet_Points[34]

In [ ]:
# Splitting the bullet points into separate columns
validation_df[['bullet_point1', 'bullet_point2', 'bullet_point3', 'bullet_point4', 'bullet_point5','bullet_point6','bullet_point7','bullet_point8','bullet_point9','bullet_point10']] = pd.DataFrame(validation_df['Cleaned_Bullet_Points'].apply(lambda x: x[:10] if len(x) >= 10 else x + [None] * (10 - len(x))).tolist())
validation_df

In [ ]:
validation_df.product_description[1572]

In [ ]:
validation_df.product_description[39]

In [ ]:
import pandas as pd
import numpy as np

def process_product_description(row):
    product_description = row['product_description']
    
    # Ensure product_description is a string
    if isinstance(product_description, str):
        # Split the text by newline character
        parts = product_description.split('\n', 1)
        
        # Extract product_description
        if len(parts) > 0:
            row['product_description'] = parts[0].strip()
        else:
            row['product_description'] = None
        
        # Extract product_specifications if exists
        if len(parts) > 1:
            product_specifications = parts[1].strip()
            # Remove "Product Specifications" from product_specifications
            product_specifications = product_specifications.replace("Product Specifications", "").strip()
            # Remove newline characters
            product_specifications = product_specifications.replace('\n', '')
            row['product_specifications'] = product_specifications
        else:
            row['product_specifications'] = None
    else:
        row['product_description'] = None
        row['product_specifications'] = None
    
    return row

# Apply the processing function to the validation_df.product_description column
validation_df = validation_df.apply(process_product_description, axis=1)
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
validation_df.head


In [ ]:
validation_df.product_specifications[1572]

In [ ]:
import pandas as pd
import re

# Function to extract key-value pairs and create new columns
def extract_specifications(row):
    if pd.notnull(row['product_specifications']):
        # Use regex to split into key-value pairs
        specifications = re.split(r'\s{2,}', row['product_specifications'].strip())
        
        for spec in specifications:
            if ':' in spec:
                key, value = spec.split(':', 1)
                key = key.strip()
                value = value.strip()
                row[key] = value
    
    return row

# Apply the extraction function to the DataFrame
validation_df = validation_df.apply(extract_specifications, axis=1)

# Display the processed DataFrame
validation_df.head


In [ ]:
validation_df.product_specifications[1572]

In [ ]:
import pandas as pd

# Define the set of headers
headers_set = {
    'Access Method', 'Age Gender Category','Is the item dishwasher safe?','Is the item microwaveable?', 'Age Range Description','Product Care Instructions', 'Allergen Information', 'Amperage', 'Animal Theme', 
    'Annual Energy Consumption', 'Assembly Instructions', 'Astm Fluid Rating', 'Athlete', 'Auto Part Position', 'Back', 
    'Baking Mix Variety', 'Band', 'Base', 'Base Type', 'Batteries Included', 'Batteries Required', 'Battery', 'Blade', 
    'Bottle Type', 'Bottom', 'Bottom Diameter', 'Bowl', 'Bowl Capacity', 'Brand', 'Brightness', 'Brim Width', 'Bristle', 
    'Bulb', 'Bullet Point', 'Cable', 'Caliber', 'California Proposition 65', 'Candy Consistency', 'Cap', 'Capacity', 
    'Capacity Name', 'Care Instructions', 'Caster', 'Center To Center Spacing', 'Certification Number', 'Chain Description', 
    'Chamber', 'Chemical Names', 'Child Parent Sku Relationship', 'Closure', 'Closure Material', 'Coarseness Rating', 
    'Coating', 'Collar Style', 'Collection', 'Collection Description', 'Color', 'Color Rendering Index', 'Color Temperature', 
    'Compatibility Options', 'Compatible Devices', 'Compatible Material', 'Compliance Media', 'Computer Memory', 
    'Configuration', 'Connectivity Protocol', 'Connectivity Technology', 'Construction Type', 'Container', 
    'Contains Liquid Contents', 'Control Method', 'Control Type', 'Controller Type', 'Controls Type', 'Cooking Surface Length Width', 
    'Core Size', 'Corner Style', 'Cost Price', 'Country As Labeled', 'Country Of Origin', 'Coverage', 'Cpsia Cautionary Statement', 
    'Cuisine', 'Current Rating', 'Customer Package Type', 'Cutting Diameter', 'Cycle Options', 'Cylinder Axis', 
    'Cylinder Correction', 'Denomination', 'Department', 'Deprecated Map Price', 'Deprecated Offering Start Date', 
    'Deprecated Shaft Style Type', 'Design Name', 'Dial', 'Diet Type', 'Digital Storage Capacity', 'Display', 
    'Display Maximum Weight Recommendation', 'Door', 'Drinking Cup Form', 'Duration', 'Each Unit Count', 'Edition', 
    'Embellishment Feature', 'Energy Consumption', 'Energy Guide', 'Energy Star', 'Eu Energy Label Efficiency Class', 
    'Exterior Finish', 'External Product Id', 'Fabric Type', 'Fabric Weight', 'Fastener', 'Fc Shelf Life', 
    'Fcc Radio Frequency Emission Compliance', 'Fill Material', 'Filter Class', 'Finish Type', 'Fit Type', 'Flavor', 
    'Folded Item Length Width Height', 'Form Factor', 'Frame', 'Furniture Finish', 'Gauge', 'Gem Type', 'Generic Keyword', 
    'Generic Type', 'Ghs', 'Giftwrap Capacity Id', 'Giftwrap Style Id', 'Giftwrap Type Id', 'Golf Club Flex', 'Golf Club Loft', 
    'Graduation Interval', 'Graphics Description', 'Grip', 'Grit', 'Gtin Exemption Reason', 'Hand Orientation', 'Handle', 
    'Hard Disk', 'Has Automatic Shutoff', 'Has Lid', 'Has Night Vision', 'Has Nonstick Coating', 'Hat Form Type', 'Hazmat', 
    'Head', 'Head Width String', 'Headband Color', 'Headwear Size', 'Heating Element Type', 'Hem Stitch Type', 'Hose Length', 
    'Hub Height', 'Identity Package Type', 'Image Locator Egus', 'Image Locator Elfl', 'Immersion Depth', 'Import Designation', 
    'Included Components', 'Included Feature', 'Indoor Outdoor Usage', 'Ink', 'Inlet', 'Inner', 'Inscription', 'Inseam', 
    'Inside', 'Installation Type', 'Insulation', 'Interior Color', 'Is Assembly Required', 'Is Autographed', 'Is Cordless', 
    'Is Customizable', 'Is Dishwasher Safe', 'Is Electric', 'Is Expiration Dated Product', 'Is Fragile', 'Is Heat Sensitive', 
    'Is Liquid Double Sealed', 'Is Microwaveable', 'Is Oem Authorized', 'Is Oven Safe', 'Is Portable', 'Is Stain Resistant', 
    'Is Trade Item Orderable Unit', 'Is Waterproof', 'Item Booking Date', 'Item Depth', 'Item Depth Width Height', 
    'Item Diameter', 'Item Dimensions','Item Dimensions W x H', 'Item Dimensions Fraction', 'Item Display Dimensions', 'Item Display Weight', 
    'Item Firmness Description', 'Item Form', 'Item Length', 'Item Length Description', 'Item Length Width', 
    'Item Length Width Height', 'Item Length Width Thickness', 'Item Name', 'Item Package Dimensions', 'Item Package Weight', 
    'Item Shape', 'Item Thickness', 'Item Torque', 'Item Type Keyword', 'Item Type Name', 'Item Volume', 'Item Weight', 
    'Item Width Diameter Height', 'Item Width Height', 'Knife Form', 'Lamp Type', 'Language', 'League Name', 
    'Legal Compliance Certifications', 'Length Range', 'Lens', 'Lifecycle Supply Type', 'Lifestyle', 'Light Color', 
    'Light Path Distance', 'Light Source', 'Lighting Method', 'Limited Edition Range', 'Line Size', 'Line Weight', 
    'Lining Description', 'Liquid Contents Description', 'Liquid Packaging Type', 'Liquid Volume', 'List Price', 
    'Lithium Battery', 'Load Capacity', 'Lock Type', 'Lower Temperature Rating', 'Magnification Strength', 
    'Main Product Image Locator', 'Manufacture Year', 'Manufacturer', 'Manufacturer Maximum Age', 'Manufacturer Minimum Age', 
    'Master Pack Layers Per Pallet Quantity', 'Master Packs Per Layer Quanti', 'Master Packs Per Layer Quantity', 'Material', 
    'Material Composition', 'Material Feature', 'Material Type Free', 'Matte Style', 'Maximum Compatible Light Source Wattage', 
    'Maximum Cut Depth', 'Maximum Power', 'Maximum Range', 'Maximum Range Indoors', 'Maximum Range Open Space', 'Maximum Size', 
    'Maximum Temperature', 'Maximum Weight Recommendation', 'Measurement Accuracy', 'Measurement System', 'Measurement Type', 
    'Measuring Range', 'Media Type', 'Melting Temperature', 'Memory Storage Capacity', 'Merchant Suggested Asin', 'Metal Type', 
    'Minimum Required Door Width', 'Model Name', 'Model Number', 'Mounting Distance', 'Mounting Type', 'Multibox', 'Neck', 
    'Net Content Area', 'Net Content Count', 'Net Content Length', 'Net Content Volume', 'Net Content Weight', 'Night Vision Range', 
    'Noise Level', 'Num Batteries', 'Number Of Blades', 'Number Of Boxes', 'Number Of Compartments', 'Number Of Heating Elements', 
    'Number Of Hooks', 'Number Of Items', 'Number Of Labels', 'Number Of Levels', 'Number Of Light Sources', 'Number Of Lithium Ion Cells', 
    'Number Of Lithium Metal Cells', 'Number Of Options Cycles', 'Number Of Pieces', 'Number Of Pockets', 'Number Of Programs', 
    'Number Of Sets', 'Number Of Settings', 'Number Of Shelves', 'Number Of Speeds', 'Number Of Trays', 'Number Of Wheels', 'Numeric Theme', 
    'Nutritional Info', 'Occasion', 'Occasion Type', 'Oem Equivalent Part Number', 'Oil Capacity', 'Opening Mechanism', 
    'Opening Shape', 'Operating Frequency', 'Operating System', 'Operation Mode', 'Optical Power', 'Other Product Image Locator 1', 
    'Other Product Image Locator 2', 'Other Product Image Locator 3', 'Other Product Image Locator 4', 'Other Product Image Locator 5', 
    'Other Product Image Locator 6', 'Other Product Image Locator 7', 'Other Product Image Locator 8', 'Outer', 'Outlet', 'Outside Diameter', 
    'Package Contains Sku', 'Package Level', 'Package Type', 'Paper Finish', 'Paper Size', 'Paper Towel Fold Type', 'Parentage Level', 
    'Part Number', 'Pattern', 'Pattern Type', 'Pesticide Marking', 'Ph Range', 'Pile Height', 'Plant Or Animal Product Type', 
    'Platform For Display', 'Ply', 'Pocket Description', 'Pore Size', 'Power Source Type', 'Processor Description', 'Product Benefit', 
    'Product Category', 'Product Expiration Type', 'Product Name', 'Product Site Launch Date', 'Product Subcategory', 'Projection Distance', 
    'Range', 'Readout Accuracy', 'Recommended Uses For Product', 'Recycled Content Percentage', 'Region Of Origin', 'Registration Status', 
    'Regulatory Compliance Certification', 'Repairability Index', 'Required Product Compliance Certificate', 'Reservoir Capacity', 
    'Resolution', 'Response Time', 'Reusability', 'Rim', 'Ring', 'Roll', 'Roll Quantity', 'Room Type', 'Rotation Period', 'Rtip Directions', 
    'Rtip Earliest Shipping Date', 'Rtip Indications', 'Rtip Ingredients', 'Rtip Inner Packs Per Master Pack', 'Rtip Items Per Inner Pack', 
    'Rtip Order Aggregate Type', 'Rtip Product Description', 'Rtip Safety Warning', 'Rug Form Type', 'Safety Data Sheet Url', 'Scent', 
    'Seasons', 'Seating Capacity', 'Seating Group', 'Septa Type', 'Series Title', 'Service Level', 'Service Plan', 'Service Provider', 
    'Serving Recommendation', 'Shade', 'Shaft', 'Shank', 'Sheet Count', 'Sheet Count Per Unit', 'Shelf', 'Shelf Thickness', 'Shelf Type', 
    'Size', 'Size Per Pearl', 'Skin Type', 'Sleeve', 'Slot Count', 'Software Included', 'Special Feature', 'Special Ingredients', 
    'Special Size Type', 'Specialty', 'Specific Uses For Product', 'Specification Met', 'Sport Type', 'Spout Outside Diameter', 
    'Sterility Rating', 'Storage Volume', 'Street Date', 'Style', 'Subject Character', 'Subject Keyword', 'Sugar Candy Type', 
    'Sun Protection', 'Supplier Declared Dg Hz Regulation', 'Support Type', 'Supported Media Size', 'Surface Recommendation', 
    'Swatch Product Image Locator', 'Switch Type', 'Target Audience', 'Target Audience Keyword', 'Target Gender', 'Target Use Body Part', 
    'Team Name', 'Temperature Range', 'Temperature Rating', 'Temperature Rating Degrees', 'Tensile Strength', 'Tension Level', 'Test Type', 
    'Theme', 'Thread', 'Tire', 'Total Diamond Weight', 'Total Eaches', 'Total Gem Weight', 'Total Metal Weight', 'Towel Form Type', 
    'Trade Size Name', 'Tray Type', 'Unit Count', 'Upper Temperature Rating', 'Variation Theme', 'Vehicle Fitment', 'Version For Country', 
    'Viewing Angle', 'Voltage', 'Volume Capacity Name', 'Waist', 'Warranty Description', 'Warranty Type', 'Water Consumption', 
    'Water Resistance Level', 'Wattage', 'Weave Type', 'Website Shipping Weight', 'Weigh Scale Type', 'Weight Capacity', 'Weight Range', 
    'Wheel', 'Wide End Diameter', 'Width Range', 'Wireless Provider', 'Zoom Ratio', 'Product Dimensions', 'Item model number', 
    'Customer Reviews', 'Best Sellers Rank', 'Date First Available', 'Bowl Material', 'Shape', 'Theme', 'Product Care Instructions', 'ASIN', 
    'Is Discontinued By Manufacturer', 'Date First Available','Special Features'
}

# Function to process product information and extract headers and values
def process_product_information(row):
    if pd.notnull(row['product_information']):
        # Clean and split the product information
        product_info = row['product_information'].split('\n')
        
        # Remove unwanted sections and clean up the data
        cleaned_info = []
        for line in product_info:
            cleaned_line = line.strip()
            if cleaned_line:
                cleaned_info.append(cleaned_line)
        
        # Extract headers and values based on the provided set of headers
        headers = []
        values = []
        i = 0
        while i < len(cleaned_info):
            header = cleaned_info[i]
            if header in headers_set:
                headers.append(header)
                if i + 1 < len(cleaned_info):
                    value = cleaned_info[i + 1].strip()
                    if value not in headers_set:
                        values.append(value)
                        i += 2
                    else:
                        values.append('')
                        i += 1
                else:
                    values.append('')
                    i += 1
            else:
                i += 1
        
        # Add extracted headers and values to the row
        for header, value in zip(headers, values):
            row[header] = value

    return row

# Apply the processing function to the DataFrame
validation_df = validation_df.apply(process_product_information, axis=1)

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Display the processed DataFrame
validation_df.head

In [ ]:
import pandas as pd
import re
# Define a function to extract the product type with a check for NaN or non-string values
def extract_product_type(text):
    if isinstance(text, str):  # Check if text is a string
        match = re.search(r"metricParameters\.productType = '(.*?)';", text)
        return match.group(1) if match else None
    return None  # Return None if the input is not a string

# Apply the function to the column and store the results in a new column
validation_df['product_type'] = validation_df['products_overview'].apply(extract_product_type)

validation_df.head

In [ ]:
import datetime

# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
file_path= f"webscrape_catalog-CA as on {current_date}.xlsx"
validation_df.to_excel(file_path, index=False)
